In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import numpy as np
import os
import pandas as pd

This code is using the Firefox webdriver to get the HTML data from the atp website. We are using a webdriver because the data were are trying to scrape is dynamically loaded, and as such doesn't appear when BeautifulSoup is used. 

In [9]:
atp_serve_url = 'https://www.atptour.com/en/stats/leaderboard?boardType=serve&timeFrame=52Week&surface=all&versusRank=all&formerNo1=false'
driver = webdriver.Firefox(executable_path=str(str(os.getcwd())+'/geckodriver'))
driver.get(atp_serve_url)

We are finding every HTML instance of the tag name 'tr', which includes the rows of data for every player in the top 80 serve leaders. These are stored in r, an array of webdriver objects

In [11]:
table = driver.find_element_by_id('leaderboardTable')
r = table.find_elements_by_tag_name('tr')
print("r length: "+str(len(r)))
print("r index 0: "+str(r[0]))

r length: 80
r index 0: <selenium.webdriver.firefox.webelement.FirefoxWebElement (session="3cfe004c-7c19-4de7-801d-d27bea3b6ef3", element="241f7270-e06a-4086-a20c-07a11eeee295")>


For every row in the atp serve leaders website, we are taking the corresponding webdriver object and taking out the name of the player and the serve rating and serve-related-data of the player, storing them in names and serve_df

In [15]:
names = []  
serve_df = []
for row in r:
  name = row.text.split('\n')[1]
  row = np.array(row.text.split('\n')[2].split(' '))
  for i in range(len(row)): 
    if '%' in row[i]: row[i] = row[i].replace('%', '')

  names.append(name)
  serve_df.append(row.astype(float))

print("names length: "+str(len(names)))
print("names first 5: "+str(names[0:5]))
print("serve_df length: "+str(len(serve_df)))
print("serve_df first 5: "+str(serve_df[0:5]))

names length: 80
names first 5: ['Hubert Hurkacz', 'Novak Djokovic', 'Stefanos Tsitsipas', 'Christopher Eubanks', 'Nicolas Jarry']
serve_df length: 80
serve_df first 5: [array([297. ,  63.9,  79.8,  51.3,  88.1,  16. ,   2.1]), array([292.7,  64.5,  77.3,  58.2,  88.7,   6.6,   2.6]), array([292. ,  64.8,  78.4,  54.9,  88.4,   7.6,   2.1]), array([289.9,  68.8,  73.8,  52.9,  85.4,  12.1,   3.1]), array([289.1,  65.1,  76.6,  53.5,  87. ,   9. ,   2.1])]


Here we are processing the names and serve_df data and putting it all into one dataframe

In [17]:
serve_df = pd.DataFrame(serve_df)
serve_df = serve_df.rename(columns={0:'serve_rating', 1:'1st_serve_%', 2:'1st_serve_points_won_%', 3:'2nd_serve_points_won_%', 
                                      4:'service_games_won_%', 5:'avg_aces_per_match', 6:'avg_double_faults_per_match'})
names = pd.Series(names)
serve_df = pd.concat([pd.Series(names), serve_df], axis=1, )
serve_df = serve_df.rename(columns={0:'name'})

serve_df.to_csv('serve_leader_df.csv')
serve_df

,name,name,serve_rating,1st_serve_%,1st_serve_points_won_%,2nd_serve_points_won_%,service_games_won_%,avg_aces_per_match,avg_double_faults_per_match
0,Hubert Hurkacz,Hubert Hurkacz,297.0,63.9,79.8,51.3,88.1,16.0,2.1
1,Novak Djokovic,Novak Djokovic,292.7,64.5,77.3,58.2,88.7,6.6,2.6
2,Stefanos Tsitsipas,Stefanos Tsitsipas,292.0,64.8,78.4,54.9,88.4,7.6,2.1
3,Christopher Eubanks,Christopher Eubanks,289.9,68.8,73.8,52.9,85.4,12.1,3.1
4,Nicolas Jarry,Nicolas Jarry,289.1,65.1,76.6,53.5,87.0,9.0,2.1
...,...,...,...,...,...,...,...,...,...
75,Botic van de Zandschulp,Botic van de Zandschulp,251.3,62.3,68.5,46.3,73.1,5.3,4.2
76,Jaume Munar,Jaume Munar,248.7,68.3,65.6,45.7,69.6,2.5,3.0
77,Sebastian Ofner,Sebastian Ofner,248.6,56.9,71.0,47.1,71.8,5.6,3.8
78,Diego Schwartzman,Diego Schwartzman,243.7,67.7,62.3,47.6,67.7,1.1,2.7
